# Fault Injection on Quantized Neural Networks

This notebook tests fault injection on quantized neural networks (QNNs). It is recommended to first read through and understand some of the example notebooks that demonstrate image classification with BNN-PYNQ.

## 1. Import the packages

In [1]:
import bnn
import os

## 2. Load the required datasets

This notebook utilizes the CIFAR-10, GTSRB, SVHN, and MNIST datasets. You can download them from each given url via wget and unzip it to a folder on Pynq as shown below.

#### CIFAR-10

In [2]:
if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/cifar-10-binary.tar.gz"):
    !wget https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz

if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/cifar-10-batches-bin/"):
    !tar -xf cifar-10-binary.tar.gz

cifar_files, cifar_labels = bnn.util.load_cifar10_testset("/home/xilinx/jupyter_notebooks/bnn/cifar-10-batches-bin/", 1000)

#### GTSRB

In [3]:
if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/GTSRB_Final_Test_Images.zip"):
    !wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_Images.zip

if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/GTSRB_Final_Test_GT.zip"):
    !wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_GT.zip

if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/GTSRB"):
    !unzip -q -o GTSRB_Final_Test_Images.zip
    !unzip -q -o GTSRB_Final_Test_GT.zip
    !mv GT-final_test.csv GTSRB/Final_Test/

gtsrb_files, gtsrb_labels = bnn.util.load_gtsrb_testset(
    "/home/xilinx/jupyter_notebooks/bnn/GTSRB/Final_Test/GT-final_test.csv",
    "/home/xilinx/jupyter_notebooks/bnn/GTSRB/Final_Test/Images",
    1000
)

--2020-09-19 20:18:00--  https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_Images.zip
Resolving sid.erda.dk (sid.erda.dk)... 130.225.104.13
Connecting to sid.erda.dk (sid.erda.dk)|130.225.104.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88978620 (85M) [application/zip]
Saving to: ‘GTSRB_Final_Test_Images.zip’

GTSRB_Final_Test_Im 100%[===================>]  84.86M  4.22MB/s    in 16s     

2020-09-19 20:18:19 (5.22 MB/s) - ‘GTSRB_Final_Test_Images.zip’ saved [88978620/88978620]

--2020-09-19 20:18:36--  https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_GT.zip
Resolving sid.erda.dk (sid.erda.dk)... 130.225.104.13
Connecting to sid.erda.dk (sid.erda.dk)|130.225.104.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99620 (97K) [application/zip]
Saving to: ‘GTSRB_Final_Test_GT.zip’

GTSRB_Final_Test_GT 100%[===================>]  97.29K   336KB/s    in 0.3s    

#### SVHN

In [4]:
if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/test_32x32.mat"):
    !wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

svhn_files, svhn_labels = bnn.util.load_svhn_testset("/home/xilinx/jupyter_notebooks/bnn/test_32x32.mat", 1000)

--2020-09-19 20:19:08--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat’

test_32x32.mat      100%[===================>]  61.30M  10.9MB/s    in 9.3s    

2020-09-19 20:19:18 (6.62 MB/s) - ‘test_32x32.mat’ saved [64275384/64275384]



#### MNIST

In [5]:
if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/t10k-images-idx3-ubyte.gz"):
    !wget http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz 
    !wget http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz 
        
if not os.path.exists("/home/xilinx/jupyter_notebooks/bnn/t10k-images-idx3-ubyte"):
    !gzip -d t10k-images-idx3-ubyte.gz
    !gzip -d t10k-labels-idx1-ubyte.gz

mnist_files, mnist_labels = bnn.util.load_mnist_testset("/home/xilinx/jupyter_notebooks/bnn/")

--2020-09-19 20:19:47--  http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 2606:4700:3036::ac43:ab4c, 2606:4700:3031::681c:6cc, 2606:4700:3033::681c:7cc, ...
Connecting to yann.lecun.com (yann.lecun.com)|2606:4700:3036::ac43:ab4c|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1648877 (1.6M) [application/x-gzip]
Saving to: ‘t10k-images-idx3-ubyte.gz’

t10k-images-idx3-ub 100%[===================>]   1.57M  7.03MB/s    in 0.2s    

2020-09-19 20:19:47 (7.03 MB/s) - ‘t10k-images-idx3-ubyte.gz’ saved [1648877/1648877]

--2020-09-19 20:19:47--  http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 2606:4700:3033::681c:7cc, 2606:4700:3031::681c:6cc, 2606:4700:3036::ac43:ab4c, ...
Connecting to yann.lecun.com (yann.lecun.com)|2606:4700:3033::681c:7cc|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4542 (4.4K) [application/x-gzip]
Saving to:

&nbsp;
## 3. A Simple Fault Test Example
This example demonstrates the most basic method of running a fault test, and demonstrates the use of all of the classification function arguments. The classes beyond this section expand on this concept and allow the user to more easily set up customizeable and comprehensive tests.

The test is performed in a way very simlar to how a normal image classification is performed. The main difference is that `classifier.classify_cifars_with_faults()` is called instead of `classifier.classify_cifars()`. This function takes additional arguments that specify the type, location, and number of faults.

For the CnvClassifier, non-CIFAR10 datasets (e.g. GTSRB and SVHN) will need to be converted to CIFAR-10 format before being classified. This will be automatically handled by using the `classifier.classify_images()` function instead of `classifier.classify_cifars()`. This function takes a list of Pillow Images and converts them to the required format before classifying. For fault injection with these datasets, use `classifier.classify_images_with_faults()`.

The inference can be performed with different precision for weights and activation. Creating a specific Classifier will automatically download the correct bitstream onto PL and load the weights and thresholds trained on the specific dataset.

&nbsp;
## 4. Using the FaultTest Class

This example demonstrates the use of the FaultTest class. This class facilitates simpler automated testing of fault injections. Two examples are provided, the latter of which shows the use of all arguments. See `bnn/faults/faults.py` for a detailed description of the FaultTest class.

&nbsp;
## 5. Using the NetworkTest Class

This example demonstrates the use of the NetworkTest class. This class allows the user to easily execute a comprehensive test of a network. See `bnn/faults/faults.py` for a detailed description of the NetworkTest class.

&nbsp;

## 6. Running Tests on All CNV Networks
The dictionary below instantiates tests for all of the available CNV networks. These tests are then conducted for each individual layer multiple times, with an increasing number of faults each time.

In [6]:
from bnn.faults import *

output_folder = "/home/xilinx/jupyter_notebooks/bnn/faults/"

num_runs = 100
flip_counts = [5, 10, 50, 100]

cnv_tests = {
    # cnvW1A1 Networks
    'cnvW1A1': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A1, cifar_files, cifar_labels)),
        NetworkTest(CNVFaultTest.SVHNTest(bnn.NETWORK_CNVW1A1, svhn_files, svhn_labels)),
        NetworkTest(CNVFaultTest.GTSRBTest(bnn.NETWORK_CNVW1A1, gtsrb_files, gtsrb_labels)),
    ],
    'cnvW1A1-TMR': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A1_TMR, cifar_files, cifar_labels)),
        NetworkTest(CNVFaultTest.SVHNTest(bnn.NETWORK_CNVW1A1_TMR, svhn_files, svhn_labels)),
        NetworkTest(CNVFaultTest.GTSRBTest(bnn.NETWORK_CNVW1A1_TMR, gtsrb_files, gtsrb_labels)),
    ],
    'cnvW1A1-interleaved': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A1_INTERLEAVED, cifar_files, cifar_labels)),
        NetworkTest(CNVFaultTest.SVHNTest(bnn.NETWORK_CNVW1A1_INTERLEAVED, svhn_files, svhn_labels)),
        NetworkTest(CNVFaultTest.GTSRBTest(bnn.NETWORK_CNVW1A1_INTERLEAVED, gtsrb_files, gtsrb_labels)),
    ],

    # cnvW1A2 Networks
    'cnvW1A2': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A2, cifar_files, cifar_labels))
    ],
#    'cnvW1A2-TMR': [
#        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A2_TMR, cifar_files, cifar_labels))
#    ],
    'cnvW1A2-interleaved': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW1A2_INTERLEAVED, cifar_files, cifar_labels))
    ],

    # cnvW2A2 Networks
    'cnvW2A2': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW2A2, cifar_files, cifar_labels))
    ],
    'cnvW2A2-TMR': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW2A2_TMR, cifar_files, cifar_labels))
    ],
    'cnvW2A2-interleaved': [
        NetworkTest(CNVFaultTest.CIFARTest(bnn.NETWORK_CNVW2A2_INTERLEAVED, cifar_files, cifar_labels))
    ],
}

In [ ]:
all_types = [
    NetworkTest.TestType.weight_bit(),
    NetworkTest.TestType.weight_word(),
    NetworkTest.TestType.threshold_bit(),
    NetworkTest.TestType.threshold_word()
]

all_tests_flat = [test for lst in cnv_tests.values() for test in lst]

for test in all_tests_flat:
    test.test_network(output_folder, num_runs, flip_counts, test_types=all_types, target_layers=[0, 1, 2, 3, 4, 5, 6, 7, 8])